In [1]:
#HIDDEN
import sys

#!pip  uninstall -y folium;
try: 
    import folium
except:
    print("hello")
    !{sys.executable} -m pip  uninstall -y folium;
    import folium

#!{sys.executable} -m pip install --user git+https://github.com/DBertazioli/folium.git
#!{sys.executable} -m pip install --user git+https://github.com/DBertazioli/folium
#!{sys.executable} -m pip install --user  -e git+https://github.com/DBertazioli/folium.git@master#egg=folium 
#!source activate py36;pip install folium --user
#!pip install --user -e git+https://github.com/DBertazioli/folium.git@master#egg=folium

import os 
import pandas as pd
import numpy as np
import datetime

try:
    import branca
except:
    print("hi")
    !{sys.executable} -m pip install branca
    import branca    

import base64 #jpeg encoding


#import folium
from folium import plugins
from folium.plugins import MarkerCluster
from folium.features import DivIcon, FeatureGroup
from folium import IFrame
import branca #brrr

In [2]:
#HIDDEN

# ## Data Prep

#to change leaflet.css: https://codepen.io/dbertazioli/pen/MdBrdV
#remember to save



## Let's build a map



template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Bacino d'utenza</li>
    <!-- questo è un commento <li><span style='background:orange;opacity:0.7;'></span>Medium</li>
    <li><span style='background:green;opacity:0.7;'></span>Small</li> -->

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""


html = """
    <h3> Event ID  : </h3><p>{}</p>
    <h3> Event Name: </h3><p></p>
    <h3> Location: </h3><p>{}</p>
    """
html1 = """
    <h2> Welcome to the meetup event map!  </h2>
    <h3> Some tips: </h3><p>{}</p>
    <h3> Some other tips: </h3><p>{}</p>
    """
html2 = """
<div style="
    position: fixed; 
    top: 80px;
    left: 80px;
    width: 300px;
    height: 340px;
    z-index:9999;
    font-size:14px;
    ">
    <h4> ID Evento: <a style="color:#e6194b;font-size:70%;">{}</a></h4>
    <h4> Event Data: </h4><p><a style="color:#e6194b;font-size:100%;">{}</a></p>
    <h4> Location: </h4><p><a style="color:#e6194b;font-size:100%;">{}</a></p>
    <h4> {} dell'evento: </h4><p><a style="color:#e6194b;font-size:100%;">{}</a></p>
    <h5> Raggio del bacino d'utenza: <a style="color:#e6194b;font-size:95%;">{} Km</a> </h5>
    <h4> Partecipanti: <a style="color:#e6194b;font-size:80%;">{}</a></h4>
    
</div>
<div style="
    position: fixed; 
    top: 70px;
    left: 70px;
    width: 320px;
    height: 360px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    filter: blur(8px);
    -webkit-filter: blur(8px);
    opacity: 0.7;
    ">
</div>
    """    

legend_html = """
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; Cool Legend <br>
     &nbsp; East &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:green”></i><br>
     &nbsp; West &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:red”></i>
      </div>
     """
legend_html2 = '''
<div style="
    position: fixed; 
    bottom: 50px;
    left: 80px;
    width: 250px;
    height: 40px;
    z-index:9999;
    font-size:14px;
    ">
    <center><p><a style="color:#e6194b;font-size:150%;">⭕</a>&emsp;Bacino d'utenza </p></center>
    <center><p><a style="color:#e6194b;font-size:200%;margin-left:20px;">⭕</a>&emsp;Bacino d'utenza </p></center>
</div>
<div style="
    position: fixed; 
    bottom: 50px;
    left: 80px;
    width: 150px;
    height: 40px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    filter: blur(8px);
    -webkit-filter: blur(8px);
    opacity: 0.9;
    ">
</div>
'''

#html_file = open('widget.html','r')
#html_file=html_file.read()
#html_file

def zoom_on_radius(radius):
    if radius >=10000:
        return 2
    if (radius >=1000 and radius <10000):
        return 3
    if (radius >= 500 and radius<1000):
        return 5
    if (radius >= 100 and radius<5000):
        return 6
    if (radius >= 50 and radius<100):
        return 8
    else:
        return 10

ddrop=pd.read_csv("https://raw.githubusercontent.com/DBertazioli/NBinteract/master/map_df.csv")

country_list=list(ddrop.Country.unique())

In [3]:
#HIDDEN

capitals={
    'fr':"Paris",
    'es':"Madrid",
    'gb':"London",
    'nl':"Amsterdam",
    'se':"Stockholm",
    'de':"Berlin",
    'cz':"Praga",
    'pl':"Varsaw",
    'ie':"Dublin",
    'be':"Bruxelles",
    'lu':"Luxemburg",
    'no':"Oslo",
    'hu':"Budapest",
    'dk':"Copenaghen",
    'ch':"Berna",
    'it':"Rome",
    'at':"Vienna",
    'fi':"Helsinki",
    'pt':"Porto",
    'ro':"Bucarest",
    'gr':"Athens",
    'ua':"Kiev",
    'hr':"Zagabria",
}
pop={
    'fr':2141000,
    'es':3174000,
    'gb':8136000,
    'nl':821752,
    'se':965232,
    'de':3575000,
    'cz':1281000,
    'pl':1765000,
    'ie':544107,
    'be':174383,
    'lu':114303,
    'no':634293,
    'hu':1756000,
    'dk':602481,
    'ch':133115,
    'it_Milano':1352000,
    'at':1868000,
    'fi':631695,
    'pt':504718,
    'ro':1836000,
    'gr':3074160,
    'it_Roma':2865000,
    'ua':2884000,
    'hr':792875
}
capital_list=[]

for country in country_list:
    try:
        capital_list.append(capitals[country])
    except:
        pass
#print(country_list)
#print(capital_list)

#HIDDEN
count_df=pd.read_csv("https://raw.githubusercontent.com/DBertazioli/NBinteract/master/count_df.csv")
#ddrop=pd.read_csv("https://raw.githubusercontent.com/DBertazioli/NBinteract/master/map_df.csv")

In [4]:
#HIDDEN

from ipywidgets import interact

#count_df=pd.read_csv("https://raw.githubusercontent.com/DBertazioli/NBinteract/master/count_df.csv")
#ddrop=pd.read_csv("https://raw.githubusercontent.com/DBertazioli/NBinteract/master/map_df.csv")
@interact(city=capital_list[:10])
def show_canned_examples(city):
    print("loading data...")
    for key, value in capitals.items():
        if value == city:
            country=key
    
    #for capitals[country] in city :
    #print(ddrop[ddrop['Country']==country]['v_lat'].mean(),ddrop[ddrop['Country']==country]['v_lon'].mean())
    m = folium.Map(location=[float(ddrop[ddrop['Country']==country]['v_lat'].mean()),float(ddrop[ddrop['Country']==country]['v_lon'].mean())], zoom_start=4, min_zoom=2)
    mc=MarkerCluster()
    radius_events = FeatureGroup(name='Events')
    for line in ddrop.itertuples():
        if line.Country==country:
            lat=line.v_lat
            lon=line.v_lon
            
            mediandist=line.mediandist
            maxdist=line.maxdist

            f = branca.element.Figure()

            m1=folium.Map([float(lat), float(lon)], zoom_start=zoom_on_radius(maxdist), min_zoom=2) #eventualmente aggiungere il livello di zoom in base al raggio può essere fyco
            Topic=str(line.topic)

            '''#enable for info in popup
            iframe = branca.element.IFrame(
                html=html.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index], Topic), 
                width=400, height=240
            )

            popup1 = folium.Popup(iframe, show=True)
            '''
            f.html.add_child(branca.element.Element('<center><h3><a href="{}" target="_blank" style="color:#e6194b;font-size:100%;">{}</a></h3></center>'.format(line.event_url,line.event_name)))
            #f.html.add_child(branca.element.Element(html_file))

            m1.add_child(folium.Circle(
              location=[float(lat), float(lon)],
              #popup=popup1,#uncomment for info in popup
              #radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
              radius=int(maxdist)*1000, #km->m
              color='crimson',
              fill=False,
              fill_color='crimson',
              fill_opacity=0.1,
              weight=0.5
            ))

            m1.add_child(folium.Circle(
              location=[float(lat), float(lon)],
              #popup=popup1,
              #radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
              radius=int(mediandist)*1000, #km->m
              color='crimson',
              fill=True,
              fill_color='crimson',
              fill_opacity=0.2,
              weight=0.1
            ))

            f.html.add_child(branca.element.Element(legend_html2))

            #html_now=html2.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index], line.count_part)
            html_now=html2.format(line.ID_event, line.date, line.venue_name, "Topics", Topic, int(mediandist), line.count_part)

            f.html.add_child(branca.element.Element(html_now))

            #f.add_child(legend)

            #f.html.add_child(branca.element.Element("<h1>Bacino d'utenza</h1>"))

            m1.add_to(f)

            iframe = branca.element.IFrame(html= '<body style= "background-color:#404040;"> </body>',width=1000, height=600)
            f.add_to(iframe)

            # Let's put the IFrame in a Popup
            popup = folium.Popup(iframe, max_width=2650)

            mc.add_child(
                folium.Marker(
                location = [float(lat), float(lon)], 
                popup=popup,
                ))
    
    radius_events.add_child(mc)        
    m.add_child(radius_events)
    plugins.Fullscreen(position='bottomright',title='Expand me', title_cancel='Exit me', force_separate_button=True
    ).add_to(m)

    #macro = MacroElement() #draggable legend just in case

    #macro._template = Template(template)

    #m.get_root().add_child(macro)
    #map.get_root().add_child(macro1)

    folium.LayerControl(position= 'bottomleft').add_to(m) #FIXME: fix position

    
    return m

interactive(children=(Dropdown(description='city', options=('Vienna', 'Bruxelles', 'Berna', 'Praga', 'Berlin',…